# OpenHT Transmit test

## Initialization

In [1]:
from openht import OpenHT

oht = OpenHT()

for item in oht.ol.ip_dict:
    print(item)

def addr(ip, offset):
    return (ip << 13) + offset*2

OpenHT successfully initialized
axi_gpio_0
openht_wrapper_0
rx_driver/rx_fifo2apb_0
tx_driver/tx_fifo2apb_0
cpu0


## OpenHT configuration
Change setup_configuration function pointer to the correct setup routine

In [2]:
def configure_tx_fm():
    pass

configure_fpga = configure_tx_fm

## GnuRadio UDP source/sink
Working at ports 10000 for input / 10001 for output

In [ ]:
from gr_src_sink import GR

gr = GR()

def get_data_from_gr():
    modulated_iq = []

    while True:
        # Get packet from GnuRadio
        tx_buffer = gr.get_in_buffer()
        tx_fixed = gr.convert_f2fixed(tx_buffer)
        
        buffer_idx = 0
        buffer_len = len(tx_fixed)
        
        while buffer_idx < buffer_len:
            # Check for FIFOs occupancy
            tx_out_count = oht.tx.read(0)
            tx_in_count = oht.openht.read(addr(0, 10)) & 0xFF
            
            # If there is enough room left in out and limited amount in input queue, send new sample
            if tx_out_count < 350 and tx_in_count <= 4:
                oht.openht.write(addr(0, 8), int(tx_fixed[buffer_idx]))
                buffer_idx += 1
        
            tx_out_count = oht.tx.read(0)
            
            # Get the modulated IQ
            for rx_idx in range(tx_out_count):
                tx_data = oht.tx.read(4)
                tx_iq = gr.convert_cplx2f(tx_data)
                if len(modulated_iq) < 350:
                    [modulated_iq.append(item) for item in tx_iq]
                else:
                    modulated_iq_f = gr.convert_fixed2f(modulated_iq)
                    gr.send_out_buffer(modulated_iq_f)
                    modulated_iq.clear()
        else:
            buffer_idx = 0

configure_fpga()
get_data_from_gr()